In [1]:
from IPython.display import Image
from PIL import Image as PIL_Image
import image_processing_aug2016 as ip
import cv2,os
import glob
reload(ip)
import json

In [12]:
# ipcluster start
#from IPython.parallel import Client
from ipyparallel import Client
rc = Client()
dview = rc[:]
dview.block=False

In [13]:
%px import geojson, numpy as np, os
%px import cv2
%px import json
!pwd
%px import image_processing_aug2016 as ip
%px cv2.__version__

/Users/cdeepakroy/work/kitware/emory/dermcli/experiments


Out[0:5]: '4.0.0'

Out[1:5]: '4.0.0'

Out[2:5]: '4.0.0'

Out[3:5]: '4.0.0'

In [14]:
dview.targets

[0, 1, 2, 3]

In [2]:
spx_ImageList = glob.glob('../sample_data/*super*.png')
### /mnt/ISIC_MIRROR_IMGSET/v2/*/*/*super*.png')
print len(spx_ImageList)
print(spx_ImageList[0])

1
../sample_data/ISIC_0000506_superpixels_v3.0.png


In [3]:
for s in spx_ImageList:
    print s
    break

../sample_data/ISIC_0000506_superpixels_v3.0.png


In [4]:
import numpy as np, cv2, geojson
### I have to switch coordinate systems, which is why I need to multiple by -1
def contourToSVGString(contour):
        """
        convert an OpenCV contour to a geojson-compatible representation
        """
        t_string = []
        for pt in contour:
            px = np.round(pt[0] )
            py =  np.round(pt[1] )  ## Appears Y is inverted/negative in the cntours
            t_string.append((float(px), float(py)))
        return t_string

# @dview.parallel(block=False)
def TileMap_to_SVG( tileImg,debug=False ):
    """
    This function takes an input image as a PNG File and produces a pointlist of the
    outer most contour for each label map, and produces a format I can read in as SVG for visualization"""
    # Load an color image in grayscale; currently not working with > 256 superpixels
    img = cv2.imread(tileImg)

    img_ch0 = img[:,:,0]
    img_ch1 = img[:,:,1]
    img_ch2 = img[:,:,2]
    
    img_minor = img_ch2  ## Need to double check this
    img_major = img_ch1   ## Also check this

    img_gray = 256 * img_major.astype(np.int) + img_minor.astype(np.int) +1 ##Allows > 256 Channels
    ### Probably should add a 1 to get rid of the 0th channel.. this is a TO DO
    
    unique_labels = np.unique(img_gray)
    if debug: print len(unique_labels)
    
    all_cnts = []
    cntdict = {}
    return_data = []
    ## So a given label/contour can contain one or more features...
    
    ## Now iterate through each label, mask the input image with the label, and find appropriate contours
    for label in unique_labels:
        working_img = img_gray.copy()
        working_img[working_img != label] = 0
        ### this breaks when label == 0
        
        ## can pass cv2.RETR_CCCOMP or RETR_EXTERNAL
        _,contours, hierarchy = cv2.findContours(working_img.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) ## can also do CHAIN_APPROX_TC89_L1
        
        if len(contours) > 0:
            if debug: print "Found %d contours for label %s" % ( len(contours), label)
            for c in contours:    
                try:
                    outerpoly = ip.contourToSVGString( np.squeeze( c ) )            
                    all_cnts.append( { 'geometry': { 'type': 'polygon', 'coordinates': outerpoly }, 'properties' : { 'labelindex': str(label-1)    } } )
                except:
                    print "SOMETHING WRONG IN THIS IMAGE CONTOUR... 2 few points??",c
        else:
            if debug: print len(contours),"were found for label",label
    for c in all_cnts:
        return_data.append(c)
        
    svg_output_file = tileImg+'.svg.json'   
    superpixel_contours = return_data
    
    if not os.path.isfile(svg_output_file):
        with  open(svg_output_file,'w') as outfile:
            json.dump(superpixel_contours, outfile)
                 
    return len(return_data)

In [5]:
## iterate through all the superpixel PNG images and see if I have a svg.json match
needsSVGJson = []
for spx in spx_ImageList:
    svg_output_file = spx+'.svg.json'
    if not os.path.isfile(svg_output_file):
        ## Make the sure the spx is not the same size as the base image
        needsSVGJson.append(spx)
        #base_image_file = spx.replace('_superpixels_v3.0.png','.jpg')
        #if os.path.isfile(base_image_file):
#             if os.path.getsize(base_image_file) ==  os.path.getsize(spx):
#                 print os.path.getsize(base_image_file), os.path.getsize(spx)
#                 print "Probably need to delete this..."
        #else:
                
            

In [6]:
print len(needsSVGJson)

ar = TileMap_to_SVG.map(needsSVGJson)

1


AttributeError: 'function' object has no attribute 'map'

In [24]:
ar.done()

True

In [7]:
res = TileMap_to_SVG(spx_ImageList[0])

ValueError: need more than 2 values to unpack

In [ ]:
# tileImg = needsSVGJson[1]
# print tileImg
# debug=True
# """
# This function takes an input image as a PNG File and produces a pointlist of the
# outer most contour for each label map, and produces a format I can read in as SVG for visualization"""
# # Load an color image in grayscale; currently not working with > 256 superpixels
# img = cv2.imread(tileImg)

# img_ch0 = img[:,:,0]
# img_ch1 = img[:,:,1]
# img_ch2 = img[:,:,2]

# img_minor = img_ch2  ## Need to double check this
# img_major = img_ch1   ## Also check this

# img_gray = 256 * img_major.astype(np.int) + img_minor.astype(np.int) +1 ##Allows > 256 Channels
# ### Probably should add a 1 to get rid of the 0th channel.. this is a TO DO

# unique_labels = np.unique(img_gray)
# if debug: print len(unique_labels)

# all_cnts = []
# cntdict = {}
# return_data = []
# ## So a given label/contour can contain one or more features...

# ## Now iterate through each label, mask the input image with the label, and find appropriate contours
# for label in unique_labels:
#     working_img = img_gray.copy()
#     working_img[working_img != label] = 0
#     ### this breaks when label == 0

#     ## can pass cv2.RETR_CCCOMP or RETR_EXTERNAL
#     _,contours, hierarchy = cv2.findContours(working_img.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) ## can also do CHAIN_APPROX_TC89_L1

#     if len(contours) > 0:
#         if debug: print "Found %d contours for label %s" % ( len(contours), label)
#         for c in contours:    
#             try:
#                 outerpoly = ip.contourToSVGString( np.squeeze( c ) )            
#                 all_cnts.append( { 'geometry': { 'type': 'polygon', 'coordinates': outerpoly }, 'properties' : { 'labelindex': str(label-1)    } } )
#             except:
#                 print "SOMETHING WRONG IN THIS IMAGE CONTOUR... 2 few points??",len(c)
#     else:
#         if debug: print len(contours),"were found for label",label
# for c in all_cnts:
# #     return_data.append(geojson.dumps(c))
#     return_data.append(c)

# svg_output_file = tileImg+'.svg.json'   
# superpixel_contours = return_data

# if not os.path.isfile(svg_output_file):
#     with  open(svg_output_file,'w') as outfile:
#         json.dump(superpixel_contours, outfile)

# return len(return_data)

In [ ]:
# _,contours, hierarchy = cv2.findContours(working_img.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) ## can also do CHAIN_APPROX_TC89_L1